## Imports

In [11]:
pip install pillow

     |████████████████████████████████| 2.2 MB 2.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
#Keras imports
import tensorflow as tf
import tensorflow.keras.preprocessing as preprocessing
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
from PIL import Image

Using TensorFlow backend.


## Pathways

In [2]:
not_path = os.listdir("../Data/Images")
not_path.remove(".DS_Store") #it likes to pop up from time to time

data_path = os.path.join('..', 'Data')

train_dir = os.path.join(data_path, 'ExModeling_train')
test_dir = os.path.join(data_path, 'ExModeling_test')
val_dir = os.path.join(data_path, 'ExModeling_val')



## vis method

In [3]:
def vis_history(history): 
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

## Modeling

In [4]:
img_size = 150
bch_size = 20

In [5]:
img_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=False, vertical_flip=False)

train_generator = img_gen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=bch_size,
    class_mode='categorical')

validation_generator = img_gen.flow_from_directory(
    val_dir, 
    target_size=(img_size, img_size), 
    color_mode='rgb',
    batch_size=bch_size, 
    class_mode='categorical')

test_generator = img_gen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    color_mode='rgb',
    batch_size=bch_size,
    class_mode='categorical',
    shuffle=False)


Found 752 images belonging to 4 classes.
Found 98 images belonging to 4 classes.
Found 92 images belonging to 4 classes.


In [ ]:
fsm = models.Sequential() 
fsm.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
fsm.add(layers.Conv2D(64, (3, 3), activation='relu'))
fsm.add(layers.MaxPooling2D((2, 2)))
fsm.add(layers.Conv2D(128, (3, 3), activation='relu'))
fsm.add(layers.MaxPooling2D((2, 2)))
fsm.add(layers.Conv2D(128, (3, 3)))
fsm.add(layers.MaxPooling2D((2, 2)))
fsm.add(layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer =tf.keras.regularizers.l1( l=0.01) ))
fsm.add(layers.MaxPooling2D((2, 2)))
fsm.add(layers.Flatten())
fsm.add(layers.Dense(64, activation='relu'))
fsm.add(layers.Dense(64, activation='relu'))
fsm.add(layers.Dense(128, activation='relu',))
fsm.add(layers.Dense(4, activation='softmax'))

fsm.compile(loss='categorical_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])

history_fsm = fsm.fit_generator(
    train_generator,
    steps_per_epoch=100,
    epochs=10, 
    validation_data=validation_generator,
    validation_steps=50)

W1016 15:11:14.761615 4629409216 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1016 15:11:14.799656 4629409216 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1016 15:11:14.806283 4629409216 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1016 15:11:14.853127 4629409216 deprecation_wrapper.py:119] From /Users/adamroth/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.m

Epoch 1/10
 84/100 [========================>.....] - ETA: 29s - loss: 49.1577 - acc: 0.3306